# <span style='color:red'>Quantitative Investing with Python</span>

### Professor Juhani Linnainmaa

Dartmouth College and Kepos Capital

*Last revised:* January 13, 2025

--- 

# **Topic 3:** CRSP Data and Short-Term Reversals

The goal in this section of the course is to **get familiarity with constructing trading strategies by replicating some academic factors**

I will use monthly CRSP data -- CRSP stands for the Center for Research in Security Prices.
- Most universities and colleges subscribe to data such as CRSP through Wharton's WRDS service
- I'm providing a part of the monthly CRSP file from May 1962 through September 2023
  - I include, at random, 25% of stocks that have at least 1 year of returns anywhere during the sample period
  
I will first consider "price-based" factors, that is, trading rules that are based only past security price information

The major price-based factors include
- Size
- Short- and long-term reversals
- Momentum
- Idiosyncratic volatility
- Betting against beta

A **factor** is just a trading rule. It specifies the rule that determines what you will buy and sell.

When you construct a factor -- as discussed in Lecture 1 -- you need to make *many* choices 
- Moreover, the data may change over time and so, in practice, it is *very* difficult to replicate a factor perfectly unless you have the original data and code
- Sometimes the original papers (and industry reports) do not provide enough details for replicating the factors
  - For example, Li, Novy-Marx, and Velikov (2019) (https://cfr.pub/published/papers/li2020liquidity.pdf) struggled to replicate a famous factor paper until they figured out that the authors had used an unreported rule:
  
  
>  "Finally, while not noted by PS, they delete zero-volume observations when estimating Eq. (1), and doing so here is crucial to generating a high correspondence between our results and those reported in their paper.
  
> Determining this fact required implementing numerous variations on the methodology described in PS. This involved labor far beyond what could reasonably be expected for casual replication, and was only possible because of the public aggregate liquidity series maintained by PS, which allowed us to infer which variations were important for generating a close correspondence." (p. 227)

In this notebook I only construct a <span style='color:red'>**short-term reversals**</span> factor

- This well-known strategy is based on the finding that stock returns display reversals at one-month horizons
  - Stocks that go up the most in one month tend to have low returns returns relative to other stocks the *next month*
  - In the academic literature the typical reference to this finding is Jegadeesh (1990): https://onlinelibrary.wiley.com/doi/abs/10.1111/j.1540-6261.1990.tb05110.x
  - Narasimhan Jegadeesh is even better known for publishing the famous 'momentum' paper three years later with Sheridan Titman<br><br>
  
- I first construct the strategy using "linear portfolio" weights to keep it simple
- I then replicate the Fama-French approach

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import requests
from io import BytesIO, StringIO
import zipfile

<br>
<div style="text-align: center; font-family: 'Georgia', sans-serif; font-size: 48px; font-weight: bold; color: red;">
    Read and process monthly stock data from CRSP
</div>

- I downloaded the CRSP data from WRDS, zipped it, and put into Dropbox
  - This is what the raw data look like
- The code below downloads the Dropbox file and unzips it into a DataFrame
- **There are lots of pre-processing steps below. We need to do this so that the data are usable.**
- Once I'm done, I'll save the processed file into JHub

In [ ]:
crsp_url = 'https://dl.dropboxusercontent.com//scl/fi/xuextjrshyajiav4wlj9b/CRSP.csv.zip?rlkey=6x8drx3htj72v7qhcppc19u5w'
response = requests.get(crsp_url)
with zipfile.ZipFile(BytesIO(response.content)) as z:
    with z.open('CRSP.csv') as f:
        df = pd.read_csv(f)
        
df.tail(3)

In [ ]:
df.T

### Pre-processing

- I don't really need company names, but I'll put them into a Series just in case I want to look something up based on PERMNO
- I also don't need TICKER

In [ ]:
company_names = df[['permno','comnam']]
company_names = company_names.groupby('permno').last().squeeze()
df = df.drop(columns=['comnam','ticker'])
company_names.name = 'Company names'
company_names.tail(5)

### Pre-processing 2

1. change dates to datetime and convert them to be as of the 1st of month
2. set permno-date as the index (it becomes a multi-index)
3. the typical universe in equities is to keep common stock traded on NYSE, Nasdaq, and AMEX -- filter based on SHRCD and EXCHCD
4. drop SHRCD because we don't need it anymore
5. convert two returns variable, DLRET and RET, into floats
   - there are some strings, which we ignore (this is the 'coerce' argument)
6. compute returns inclusive of delisting returns
   - our return variable is either 'normal return', 'delisting return', or, if both exist, the compounded return
   - we need to be careful with missign values - I'm filling in zeros for NaNs in the computation but then, if neither return exists, putting them back in  
7. compute market capitalization in millions
   - note that PRC is negative to indicate that it is the spread midpoint

In [ ]:
# convert the data variable and put permno and date in the index
df['date'] = pd.to_datetime(df['date'], format='%d%b%Y').dt.to_period('M').dt.to_timestamp()
df = df.set_index(['permno', 'date']).sort_index()

# we want to keep SHRCD = 10 or 11 and EXCHCD=1,2, or 3 -- the raw input data should already have these filters
df = df[(df['shrcd'].isin([10,11])) & (df['exchcd'].isin([1,2,3]))]

# drop the SHRCD variable - we don't need it anymore
df = df.drop(columns=['shrcd'])

# what *is* DLRET?
print(df['dlret'].describe())

col_list = ['dlret', 'ret']
for col in col_list:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    
df['ret2'] = (1 + df['ret'].fillna(0)) * (1 + df['dlret'].fillna(0)) - 1

neither_return_exists = (df['ret'].isnull()) & (df['dlret'].isnull())
df.loc[neither_return_exists, 'ret2'] = np.nan

# drop the original return variables - we don't need them anymore
df = df.drop(columns=['ret', 'dlret'])
df = df.rename(columns={'ret2': 'ret'})

# compute market cap in millions
df['me'] = np.abs(df['prc']) * df['shrout'] / 1_000
df['me'] = df['me'].replace({0: np.nan}) # Shares outstanding is sometimes zero or PRC is missing -> set me to missing
df = df.drop(columns=['prc', 'shrout']) # Drop PRC -> we don't need it anymore

df.tail(10)

## There are a few duplicate observations: same permno and month

- Let's get rid of them by averaging.

In [ ]:
print(f'Is the index unique? {df.index.is_unique}')
df = df.groupby(level=['permno', 'date']).mean()
print(f'Is the index now unique? {df.index.is_unique}')

## Which observations have the biggest market capitalization in our data?

In [ ]:
df.sort_values('me', ascending=False).head(10)

Which stock had the market cap of \\$513,362M in September 2007?

In [ ]:
company_names.loc[11850]

### Save finished file into a pickle (```PKL```) file

- Be careful with pickle files - they are very convenient but not efficient and break between Python and Pandas versions!

Let me try to figure out the full path to the root directory; it has a weird name; for me it is ```/home/jovyan```

- I want to save the data in a ```data``` directory below the root
- ```os``` package has all kinds of functions for dealing with the filesystem

In [ ]:
import os

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

print(f'Current directory: {current_dir}')
print(f'Parent directory: {parent_dir}')

In [ ]:
# Specify the directory and filename (CHANGE AS NEEDED!)
directory = '/home/jovyan/data'
filename = 'crsp.pkl'

# Create the directory if it doesn't exist
os.makedirs(directory, exist_ok=True)

# Save the DataFrame as a pickle file
file_path = os.path.join(directory, filename)
df.to_pickle(file_path)

<br>
<div style="text-align: center; font-family: 'Georgia', sans-serif; font-size: 48px; font-weight: bold; color: red;">
    Short-Term Reversals Strategy
</div>

## Trading strategies

A trading strategy is a systematic rule based on data we know at the time we make the trading decisions

Your strategy could be **value investing:**

1. Buy value stocks and sell growth stocks (this is known as a long-short portfolio)
2. Look at the data once a month to rebalance your portfolios as stocks' characteristics change

### Strategy 1: Short-term reversals

**Short-term reversals** is the empirical finding that, at short horizons, stock returns tend **reverse**

In monthly data, a strategy that trades short-term reversals is simple:

- Rank stocks by their prior-month returns (e.g., their returns in December)
- At the end of the month, buy stocks with the lowest returns and sell stocks with the highest returns

To get started, I'll create the following strategy:

- Every month rank stocks based on their one-month returns
- Buy stocks with the lowest returns and sell those with the highest returns

We need to pay some attention to timing

## Compute portfolio weights

- We want to invest the most in stocks with the most negative returns and short the most those with the most positive returns
- I'll rank stocks each month based on returns. 
  - I don't use ```rank(pct=True)``` because it runs from 1/N to 1 (instead of 0 to 1 as I want)
  - I do ```* (-2) + 1``` to give a +100% weight to the stock with the lowest return and a weight of -100% to the stock with the highest return
  - *Note:* The scale of my weights here is arbitrary. It won't make sense to invest -100% to one stock and 95% in another -- I'll address this issue later

In [ ]:
# reload the data
df = pd.read_pickle('/home/jovyan/data/crsp.pkl')

df['position'] = (
    df.groupby(level='date')['ret']  # Group by the 'date' level in the index
    .transform(lambda x: (x.rank() - x.rank().min()) / (x.rank().max() - x.rank().min()) * (-2) + 1)
)

## Compute portfolio return

- Let me keep this simple and create two different dataframes:

1. ```Position``` will have the weights we just computed
2. ```Ret``` will have the asset returns

I reshape these so that we have ```date``` in the index and ```permno``` in the columns

In [ ]:
position = df['position'].unstack(level='permno')
ret = df['ret'].unstack(level='permno')

display('position:', position.tail(5))
display('ret:', ret.tail(5))

**Remember** that, at the moment, position on the "November 2022" row is based on November 2022 returns

- We want to take this position at the end of November 2022 and earn the return on this position in December 2022
- So we need to **shift** weights back by one month

In [ ]:
lagged_position = position.shift(1)

### The return on an asset is just the product of weights and returns

- Once we have those products, we can just take the sum across all assets to get the total return per month

In [ ]:
portfolio_return = lagged_position.mul(ret).sum(axis=1, min_count=1)
portfolio_return.tail(10)

### These don't look like returns because our weights are far too large (we are investing -100% in the stock with the highest return and +100% in the stock with the lowest return

- I can figure out how to scale these weights by looking at these returns
- For example, if it looks like the portfolio has realized a volatility of 100% and I want to target a volatility of 15%, I can just multiple the weights by (15% / 100%)
- What I do is, I look at the realized volatility over the prior 12 months (up to one month ago) and use that to target volatility
  - There is no lookahead bias
- I'll call the resulting strategy ```strev```, which is the typical abbreviation for short-term reversals

In [ ]:
realized_vol = np.sqrt(12) * portfolio_return.rolling(window=12, min_periods=3).std()
scaled_position = 0.15 * position.div(realized_vol.shift(1), axis=0) 
scaled_lagged_position = scaled_position.shift(1)

strev = scaled_lagged_position.mul(ret).sum(axis=1, min_count=1)
strev.tail(10)

### How much volatility do we realize?

In [ ]:
display(strev.describe())
print(f'\nAnnualized vol: {np.sqrt(12) * strev.describe()["std"]:.2%}')

### Analyze the strategy's performance

I define a helper function for doing some analysis
- It just means that I don't have to rewrite the same code
- It is good to write modular code

In [ ]:
def analyze_returns(r=None, name=None, start_date='1964-01', end_date='2023-09'):
    r = r.loc[start_date:end_date]
    ir = np.sqrt(12) * r.mean() / r.std()
    print(f'Start: {start_date}, End: {end_date}')
    print(f'Sharpe ratio: {ir:.2f}')
    r.cumsum().plot(title=f'Analysis of a strategy: "{name}"', figsize=(12,8))

In [ ]:
analyze_returns(strev, 'Short-term reversals (linear weights)', end_date='1995-12')

## How does this strategy work?

The process of running a quantitative strategy looks like this:

1. Update and verify data
2. Regenerate the signal / portfolios (that is, the weights)
   - With short-term reversals, we need up-to-date returns for computing the new positions
3. Get today’s portfolio values
4. Query your current portfolio and prices
5. Compute your new portfolio as ```portfolio value * weights```
6. Your trades will be ```new portfolio``` - ```old portfolio```
   - Convert to shares if needed

If your NAV is $1M, these would be your top and bottom positions at the end of December 2000

In [ ]:
AUM = 1_000_000

dollar_positions = scaled_position.loc['2000-12'].squeeze().dropna() * AUM
dollar_positions.sort_values().round(2)

Your gross exposure would be

In [ ]:
print(f'Your gross exposure is {dollar_positions.abs().sum():,.2f} dollars, which means that your gross leverage is {dollar_positions.abs().sum() / AUM:.2f}')

## Notes on leverage

Your gross (or "two-sided") leverage is less than 1.0 because you're running a relative safe portfolio (targeting only 15% vol). 

If you were to increase the targeted vol, this number would creep up. 

For example, doubling the target volatility would double leverage.

## Trading 

How much would you trade at the end of January 2001 when your signals and portfolio have changed?

In [ ]:
dollar_positions = scaled_position.loc['2000-12':'2001-01'].T.dropna(how='all').fillna(0) * AUM
dollar_positions.columns = ['old portfolio', 'new portfolio']
dollar_positions['trades'] = dollar_positions['new portfolio'] - dollar_positions['old portfolio']
dollar_positions.sort_values('trades').round(2)

## Our computation above ignores trading costs

- There are two types of trading costs:
  1. Bid-ask spread (you need to pay this no matter what)
  2. Price impact (the more you trade the more it'll cost you)
- We can assume that other fees (exchange fees, commissions,...) are effectively zero

When we run quantitative strategies, our goal is to maximize performance *net* of trading costs
- This can make a big difference when you consider faster strategies
- You pay trading costs when you demand liqudity; HFTs typically make money by *providing* liquidity -- you can think of their trading costs as being negative

We would need to have estimates of trading costs
- Modeling trading costs is really difficult!
- There are academic papers on the topic plus brokers can provide their own estimates

Let's suppose that our trading costs are flat 0.3% of the trade value (they are lower for liquid stocks and much higher for illiquid stocks)

What is the math?

- If you buy ```$1,000``` worth of a stock, you pay 0.3\% * ```$1,000``` = ```$1``` in trading costs
- We need to compute the change in weights. In terms of weights, the cost you pay is |new weight - old weight| * 0.3%.
  - We need to subtract this from returns

In [ ]:
trades = scaled_lagged_position.fillna(0) - scaled_lagged_position.fillna(0).shift(1)
abs_trades = trades.abs().sum(axis=1)
tcost = abs_trades * 0.003

print('Recent trading costs (%) from rebalancing the portfolio:')
display((100 * tcost).tail(5).round(2))

In [ ]:
strev_net = strev - tcost

analyze_returns(strev_net, 'Short-term reversals (linear weights, net of trading costs)', end_date='1995-12')

## Compare to Fama and French's computation

- We want to make sure that we did this (approx.) right
  - As we will see, there is a small discrepancy

### Get Fama-French factors from Ken French's website 

- I write a helper function ```download_french_data``` that I can use to download the data
- There are some file-specific issues that I need to control for 
  - In the French data files, there are varying amounts of additional lines of text at the top and the same file often has multiple datasets (e.g., monthly and annual factors)
    - In my code below, I lightly control for these issues at least for the two files I want to download
- I get both Fama-French factors and portfolios formed based on short-term reversals

In [ ]:
def download_french_data(url=None, csvname=None, skiplines=None):
    
    response = requests.get(url)

    # If the request is NOT successful, raise an exception
    if response.status_code != 200:
        raise Exception(f"Failed to download zip file. Status code: {response.status_code}")

    with zipfile.ZipFile(BytesIO(response.content)) as zip_file:

        # Check if the file exists in the zip archive
        if csvname in zip_file.namelist():
            # Read the CSV file directly from the zip archive
            with zip_file.open(csvname) as csv_file:
                lines = csv_file.readlines()

            # Remove rows from the beginning
            lines = lines[skiplines:]

            # Create a DataFrame from the trimmed lines using StringIO
            # First need to decode byte strings into unicode
            lines = [line.decode("utf-8") for line in lines]

            # at some point the file switches from monthly factors to annual factors and other stuff
            # we can delete what ever comes after
            for idx, line in enumerate(lines):
                if ('Annual Factors' in line) or (len(line.strip())==0): break
                
            lines = lines[:idx]
            clean_csv = '\n'.join(lines)
            df = pd.read_csv(StringIO(clean_csv))   
            
            # convert date into a format we understand and make it the index
            # also convert returns from percentages (e.g., 2.12) to decimles (e.g., 0.0212) by dividing by 100
            df['date'] = df['Unnamed: 0'].apply(lambda x: datetime.strptime(str(x), '%Y%m'))
            df = df.drop(columns='Unnamed: 0')
            df = df.set_index('date') / 100

            print(f'File {csvname} read successfully!')
            return df
        else:
            print(f'Zip file found but file {csvname} not found in the archive.')   
            return pd.DataFrame()

### Read FF5 factors and clean the data

- Convert returns to decimals and date from YYYYMM to datatime

In [ ]:
# Specify the file we want to read -- the CSV file inside has almost the same name 
url = 'https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_5_Factors_2x3_CSV.zip'
csvname = 'F-F_Research_Data_5_Factors_2x3.csv'

ff_data = download_french_data(url=url, csvname=csvname, skiplines=3)
ff_data.to_pickle('../data/ff_data.pkl')

print('\nData:\n')
display(ff_data.head(3))

### Read returns on decile portfolios formed based on short-term reversals

- This is similar to what we were doing above, except that French reports returns for all ten portfolios
  - We'll take them all and then compute the return on 'losers' minus 'winners'
- Similar to above, after we get the raw data, I clean it a bit

In [ ]:
url = 'https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_ST_Reversal_Factor_CSV.zip'
csvname = 'F-F_ST_Reversal_Factor.CSV'    

strev_ff = download_french_data(url=url, csvname=csvname, skiplines=13)

print('\nData after processing:\n')
display(strev_ff.head(3))

# convert to a Series (it was a DataFrame)
strev_ff = strev_ff.squeeze()

In [ ]:
analyze_returns(strev_ff, 'Short-term reversals (FF version)', end_date='1995-12')

### Correlation between our strategy and that of Fama and French

In [ ]:
ours = strev.copy()
ours.index = ours.index.to_period('M').to_timestamp('M')
ours.name = 'Our strategy'

theirs = strev_ff.copy()
theirs.index = theirs.index.to_period('M').to_timestamp('M')
theirs.name = 'FF\'s strategy'
pd.concat([ours, theirs], axis=1).corr().round(3)

### <span style='color:red'> **Note:**</span> Our replication does not match the Fama-French return exactly

**Question: What is the difference?**

We are doing a few things differently:

1. Fama and French construct their strategy by sorting stocks into six portfolio, making these portfolios value-weighted, and then are long and short these portfolios
   - Fama and French also put their stocks into portfolios using "NYSE breakpoints" 
2. Our weights are proportional to realized returns
3. Fama and French include all stocks; we include just 25% selected at random

<br>
<div style="text-align: center; font-family: 'Georgia', sans-serif; font-size: 48px; font-weight: bold; color: red;">
    A replication of Fama and French's factor
</div>

## 1. Compute the NYSE breakpoints for size and 1-month return
   
- 50th percentile for market cap (size)
- 30th and 70th percentiles for return

**Note:** I keep only ```exchcd==1``` observations in the sample and then group by month

In [ ]:
df = pd.read_pickle('/home/jovyan/data/crsp.pkl')

grp = df.loc[df['exchcd']==1]['me'].dropna().groupby(level='date')
me_p50 = grp.apply(lambda x: np.percentile(x, 10))

grp = df.loc[df['exchcd']==1]['ret'].dropna().groupby(level='date')
ret_p30 = grp.apply(lambda x: np.percentile(x, 30))
ret_p70 = grp.apply(lambda x: np.percentile(x, 70))

breakpoints = pd.DataFrame(
    {'me_p50': me_p50,
     'ret_p30': ret_p30, 
     'ret_p70': ret_p70,}
)

breakpoints.tail(5)

### Merge the breakpoints back into our original dataframe

- We need to specify what we merge on
- On the "left" we are merging by (level) "date"; on the "right" we are merging by the index (which is also date)
- We also need to specify what observations we want to keep: those on the left, those on the right, the union (inner), or the join (outer)

In [ ]:
df = df.merge(breakpoints, left_on='date', right_index=True, how='left')
df.tail(5)

## Assign stocks into portfolios based on where they fall based on the breakpoints

- For Fama and French, we only need the losers (<= 30th percentile) and winners (> 70th percentile)

In [ ]:
df['portfolio'] = '' # initialize the column (not super-necessary)
df.loc[(df['ret'] <= df['ret_p30']) & (df['me'] <= df['me_p50']), 'portfolio'] = 'small_losers'
df.loc[(df['ret'] <= df['ret_p30']) & (df['me'] >  df['me_p50']), 'portfolio'] = 'big_losers'
df.loc[(df['ret'] >  df['ret_p70']) & (df['me'] <= df['me_p50']), 'portfolio'] = 'small_winners'
df.loc[(df['ret'] >  df['ret_p70']) & (df['me'] >  df['me_p50']), 'portfolio'] = 'big_winners'

## Compute returns for the value-weighted portfolios

- Let's pretend that I invest 'me' into each stock at the end of the month
- If I want to compute the return on the portfolio the next month, it'll be something like ```sum(me * return) / sum(me)``` -- how much money I made divided by the total amount I invested
- To get the March 1986 return, I need to get portfolio assignments and market caps from February 1986
  - This is the timing stuff again
  - **Important:** If I have multiples levels in the index, ```shift()``` wouldn't probably do what you want
  - It would just take the value from the previous row -- might be for a different stock
  - Do make sure that I grab the right return, I need to ```groupby``` permno

In [ ]:
df

In [ ]:
df['Lme'] = df.groupby('permno')['me'].shift(1)
df['Lportfolio'] = df.groupby('permno')['portfolio'].shift(1)

df['Lme_x_ret'] = df['Lme'].mul(df['ret'])

sums = df.reset_index().groupby(['date', 'Lportfolio'])[['Lme', 'Lme_x_ret']].sum(min_count=1)
portfolio_returns = sums['Lme_x_ret'].div(sums['Lme']).unstack(level='Lportfolio')
portfolio_returns

## Compute the return for the Fama-French-style factor 

In [ ]:
strev_vw = (
    (1/2) * (portfolio_returns['small_losers'] + portfolio_returns['big_losers']) -
    (1/2) * (portfolio_returns['small_winners'] + portfolio_returns['big_winners'])
)

analyze_returns(strev_vw, 'Short-term reversals (our FF replication)', end_date='1995-12')

## Comparison between the three strategies

- The remaining difference is about us having only 25% of the CRSP universe

In [ ]:
ours = strev.copy()
ours.index = ours.index.to_period('M').to_timestamp('M')
ours.name = 'Linear'

our_vw = strev_vw.copy()
our_vw.index = our_vw.index.to_period('M').to_timestamp('M')
our_vw.name = 'VW'

theirs = strev_ff.copy()
theirs.index = theirs.index.to_period('M').to_timestamp('M')
theirs.name = 'FF'
pd.concat([ours, our_vw, theirs], axis=1).corr().round(3)

## Short-term reversals with trading costs

- Let me compute the portfolio weights, which are proportional to market values of equity within each 'sub-portfolio' (e.g., small_winners)

In [ ]:
df['w_portfolio'] = (
    df['Lme'] / 
    df.groupby([df.index.get_level_values('date'), 'Lportfolio'])['Lme'].transform('sum')
)

multipliers = {
    'small_winners': -(1/2),
    'small_losers':   (1/2),
    'big_winners':   -(1/2),
    'big_losers':     (1/2)
}

df['w'] = df['Lportfolio'].map(multipliers) * df['w_portfolio']

weights = df['w'].unstack(level='permno').fillna(0)

In [ ]:
abs_trades = (weights - weights.shift(1)).abs().sum(axis=1)
abs_trades
tcost = abs_trades * 0.003

print('Recent trading costs (%) from rebalancing the portfolio:')
display((100 * tcost).tail(5).round(2))

In [ ]:
strev_vw_net = strev_vw - tcost 

analyze_returns(strev_vw_net, 'Short-term reversals (our FF replication, net of t-costs)', end_date='1995-12')